In [1]:
import numpy as np
import pandas as pd
import pandas_profiling as pp
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

2021-08-30 03:11:16.077202: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-30 03:11:16.077245: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
raw_df = pd.read_csv('../../../test-builds/llvm/datasets/model-aug-29.csv')
raw_df.head()

,fullpath,modulepath,function,BasicBlockCount,BasicBlockWithSingleSuccessor,BasicBlockWithTwoSuccessors,BasicBlockWithMoreThanTwoSuccessors,BasicBlockWithSinglePredecessor,BasicBlockWithTwoPredecessors,BasicBlockWithMoreThanTwoPredecessors,...,CastInstCount,FloatingPointInstCount,IntegerInstCount,LoadInstCount,StoreInstCount,CallInstCount,PHIInstCount,AllocaInstCount,GEPInstCount,label
0,/mnt/disks/data/tarindu/test-builds/llvm/code-...,/mnt/disks/data/tarindu/test-builds/llvm/code-...,_GLOBAL__sub_I_APFixedPointTest.cpp,100,55,33,11,55,33,11,...,110,0,0,110,176,222,0,44,143,O2
1,/mnt/disks/data/tarindu/test-builds/llvm/code-...,/mnt/disks/data/tarindu/test-builds/llvm/code-...,_ZN10llvm_gtestlsERSoRKNS_14RawStreamProxyIA1...,7,3,3,0,4,1,1,...,5,0,1,4,8,7,1,1,12,FNF
2,/mnt/disks/data/tarindu/test-builds/llvm/code-...,/mnt/disks/data/tarindu/test-builds/llvm/code-...,_ZN10llvm_gtestlsERSoRKNS_14RawStreamProxyIN4...,3,1,1,0,1,1,0,...,5,0,0,4,7,9,0,2,11,O1-O2-O3-FNF
3,/mnt/disks/data/tarindu/test-builds/llvm/code-...,/mnt/disks/data/tarindu/test-builds/llvm/code-...,_ZN10llvm_gtestlsERSoRKNS_14RawStreamProxyIPKcEE,7,3,3,0,4,1,1,...,5,0,1,5,8,7,1,1,11,O1-O2-O3-FNF
4,/mnt/disks/data/tarindu/test-builds/llvm/code-...,/mnt/disks/data/tarindu/test-builds/llvm/code-...,_ZN12_GLOBAL__N_112CheckIntPartERKN4llvm19Fix...,148,65,74,0,95,38,14,...,86,0,48,94,53,147,4,33,92,O2


In [3]:
raw_df['label'].describe()

count            178012
unique               11
top        O1-O2-O3-FNF
freq              84434
Name: label, dtype: object

### Clean Dataset

In [4]:
cleaned_df = raw_df.copy()

# don't want the 'fullpath', 'modulepath', 'function' columns
cleaned_df = cleaned_df.drop(['fullpath', 'modulepath', 'function'], axis=1)

def clean_labels(x):
    if x.strip() == 'O1':
        return 1 
    elif x.strip() == 'O2':
        return 2 
    elif x.strip() == 'O3':
        return 3
    else:
        return 0
    

cleaned_df['label'] = cleaned_df['label'].map(clean_labels)

cleaned_df.head()        

,BasicBlockCount,BasicBlockWithSingleSuccessor,BasicBlockWithTwoSuccessors,BasicBlockWithMoreThanTwoSuccessors,BasicBlockWithSinglePredecessor,BasicBlockWithTwoPredecessors,BasicBlockWithMoreThanTwoPredecessors,BigBasicBlock,MediumBasicBlock,SmallBasicBlock,...,CastInstCount,FloatingPointInstCount,IntegerInstCount,LoadInstCount,StoreInstCount,CallInstCount,PHIInstCount,AllocaInstCount,GEPInstCount,label
0,100,55,33,11,55,33,11,0,22,78,...,110,0,0,110,176,222,0,44,143,2
1,7,3,3,0,4,1,1,0,1,6,...,5,0,1,4,8,7,1,1,12,0
2,3,1,1,0,1,1,0,0,1,2,...,5,0,0,4,7,9,0,2,11,0
3,7,3,3,0,4,1,1,0,1,6,...,5,0,1,5,8,7,1,1,11,0
4,148,65,74,0,95,38,14,0,7,141,...,86,0,48,94,53,147,4,33,92,2


In [ ]:
pp.ProfileReport(cleaned_df)

Summarize dataset:   0%|          | 0/41 [00:00<?, ?it/s]

### Split the dataframe into train, validation, and test

In [ ]:
train, test = train_test_split(cleaned_df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')